In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [2]:
import multiprocessing

import pandas as pd
import datamol as dm
import numpy as np
import seaborn as sns

from molfeat.calc import RDKitDescriptors2D, FPCalculator, MordredDescriptors
from molfeat.trans import MoleculeTransformer
from sklearn.preprocessing import OneHotEncoder

import collections.abc as collections
from molfeat.trans.concat import FeatConcat

from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from src.config import mem
from src.utils import embed3d, eval_model
from src.descriptors import get_dgl_predictions, get_hft_predictions

from rdkit import RDLogger, Chem

RDLogger.DisableLog('rdApp.*')

In [3]:
train = dm.read_csv("../data/processed/train.csv", smiles_column="smi", index_col=0)
test = dm.read_csv("../data/processed/test.csv", smiles_column="smi", index_col=0)
y_train = pd.read_pickle('../data/processed/y_train.pkl')

In [4]:
ohe = OneHotEncoder(sparse_output=False)

def get_x_train(feats):
    return np.concatenate([feats, ohe.fit_transform(train[["prop"]])], axis=1)

def get_x_test(feats):
    return np.concatenate([feats, ohe.transform(train[["prop"]])], axis=1)

In [6]:
from molfeat.trans.fp import FPVecTransformer
from molfeat.trans.pretrained import PretrainedDGLTransformer
from molfeat.trans.pretrained.hf_transformers import PretrainedHFTransformer


dgl_params = [
    {'kind': 'gin_supervised_contextpred'},
    {'kind': 'gin_supervised_infomax'},
    {'kind': 'gin_supervised_edgepred'},
    {'kind': 'gin_supervised_masking'},
]

hft_params = [
    # {'kind': 'MolT5', 'notation': 'smiles'},
    {'kind': 'GPT2-Zinc480M-87M', 'notation': 'smiles', 'random_seed': 42},
    # {'kind': 'Roberta-Zinc480M-102M', 'notation': 'smiles', 'random_seed': 42},
    
]

transformers = [    
    FPVecTransformer("ecfp:4", length=1024, n_jobs=1, dtype=np.float32),
    FPVecTransformer("maccs", length=1024, n_jobs=1, dtype=np.float32),
    FPVecTransformer("topological", length=1024, n_jobs=1, dtype=np.float32),
    FPVecTransformer("avalon", length=1024, n_jobs=1, dtype=np.float32),
    FPVecTransformer('erg', length=315, dtype=np.float32),    
    FPVecTransformer("layered", n_jobs=1, length=1024, dtype=np.float32),
    FPVecTransformer("secfp", length=1024, n_jobs=1, dtype=np.float32),
    FPVecTransformer("estate", n_jobs=1, dtype=np.float32),
    FPVecTransformer('pattern', length=1024, dtype=float),
    
    FPVecTransformer("mordred", n_jobs=-1, dtype=np.float32),
    FPVecTransformer("desc2D", n_jobs=-1, dtype=np.float32, replace_nan=True),
    FPVecTransformer("cats2D", n_jobs=-1, dtype=np.float32, replace_nan=True),    
    FPVecTransformer("pharm2D", n_jobs=-1, length=1024, dtype=np.float32),    
    FPVecTransformer("scaffoldkeys", n_jobs=-1, dtype=np.float32, replace_nan=True),
    FPVecTransformer("skeys", n_jobs=-1, dtype=np.float32, replace_nan=True),
    
    # PretrainedDGLTransformer('gin_supervised_contextpred', dtype=float),
    # PretrainedDGLTransformer('gin_supervised_infomax', dtype=float),
    # PretrainedDGLTransformer('gin_supervised_edgepred', dtype=float),
    # PretrainedDGLTransformer('gin_supervised_masking', dtype=float),
    # 
    # PretrainedHFTransformer('MolT5', notation='smiles', dtype=float),
    # PretrainedHFTransformer('GPT2-Zinc480M-87M', notation='smiles', dtype=float),
    # PretrainedHFTransformer('Roberta-Zinc480M-102M', notation='smiles', dtype=float),
]

transformers3d = [
    FPVecTransformer("desc3D", length=639, dtype=np.float32, replace_nan=True),
    FPVecTransformer("cats3D", length=126, dtype=np.float32, replace_nan=True),
    FPVecTransformer("pharm3D", length=1024, dtype=np.float32),
    FPVecTransformer("electroshape", length=15, dtype=np.float32, replace_nan=True),
    FPVecTransformer("usr", length=12, dtype=np.float32),
    FPVecTransformer("usrcat", length=60, dtype=np.float32),
]

# featurizer = FeatConcat(transformers, dtype=np.float32)

# calcucalte feats and cache them
for trans in transformers:    
    feats = mem.cache(trans.transform)(train.smi)
    
for params in dgl_params:
    feats = mem.cache(get_dgl_predictions, ignore=['n_jobs', 'dtype'])(train.smi, params)    

for params in hft_params:
    feats = mem.cache(get_hft_predictions, ignore=['n_jobs', 'dtype', 'device'])(train.smi, params, device='cpu', n_jobs=-1)
    
# for kind, params in hft_params.items():
#     print('Initializing', end=' ')    
#     trans = PretrainedHFTransformer(kind, **params, n_jobs=-1)
#     print(kind, end=' ')
#     feats = mem.cache(trans.transform)(train.smi)
#     print(feats.shape[1])

________________________________________________________________________________
[Memory] Calling src.descriptors.get_hft_predictions...
get_hft_predictions(id
0            O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2
1       O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...
2                               [N-]=[N+]=CC(=O)NCC(=O)NN
3                               [N-]=[N+]=C1C=NC(=O)NC1=O
4               CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O
                              ...                        
7934       O=c1[nH]c2cc(Cl)c(Cl)c([N+](=O)[O-])c2[nH]c1=O
7935    C[S+](CCC(N)C(=O)[O-])CC1OC(n2cnc3c(N)ncnc32)C...
7936               CC(Cc1ccccc1)N1CC(=NC(=O)Nc2ccccc2)ON1
7937    CCc1c(C)[n+]([NH-])c(-c2ccc(OC)c(OC)c2)c2cc(OC...
7938    [N-]=[N+]=NCC(=O)NC(CO)C(O)c1ccc([N+](=O)[O-])cc1
Name: smi, Length: 7939, dtype: object, 
{'kind': 'GPT2-Zinc480M-87M', 'notation': 'smiles', 'random_seed': 42}, device='cuda')


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.63 GiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 35.73 GiB is allocated by PyTorch, and 48.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
results = {}
clf = HistGradientBoostingClassifier(random_state=42)
train_all_feats = np.empty((train.shape[0], 0))

for trans in transformers:
    feats = mem.cache(trans.transform)(train.smi)
    X_train = get_x_train(feats)    
    label = f"{trans.kind} {trans.length}"
    results[trans.kind] = eval_model('', clf, X_train, y_train)
    train_ft = np.hstack([train_all_feats, feats])        

sns.boxplot(pd.DataFrame(results))

 ecfp:4:1024: 0.8717    (0.881 ± 0.009)    9.3s
  maccs:1024: 0.8825    (0.891 ± 0.009)    2.1s
topological:1024: 0.8627    (0.870 ± 0.007)    9.1s
 avalon:1024: 0.8866    (0.896 ± 0.009)    9.7s
     erg:315: 0.8627    (0.871 ± 0.008)    3.0s
layered:1024: 0.8857    (0.893 ± 0.007)    10.2s
  secfp:1024: 0.8755    (0.883 ± 0.008)    9.9s
 estate:2000: 0.8715    (0.876 ± 0.005)    1.7s
________________________________________________________________________________
[Memory] Calling molfeat.trans.base.MoleculeTransformer.transform...
transform(id
0            O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2
1       O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...
2                               [N-]=[N+]=CC(=O)NCC(=O)NN
3                               [N-]=[N+]=C1C=NC(=O)NC1=O
4               CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O
                              ...                        
7934       O=c1[nH]c2cc(Cl)c(Cl)c([N+](=O)[O-])c2[nH]c1=O
7935    C[S+](CCC(N)C(=O)[O-])CC1OC(n2cnc3

  0%|          | 0.00/636 [00:00<?, ?B/s]

  0%|          | 0.00/7.12M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [63]:
# train['mol3d'] = mem.cache(embed3d)(train.smi, n_confs=None)

In [94]:
for trans in transformers3d:    
    results = trans.transform(train.mol3d, ignore_errors=True)
    if isinstance(results, list):
        # replace None with np.array of np.nan
        res = [a if a is not None else np.full(trans.length, np.nan) for a in results]        
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
        results = imp.fit_transform(np.stack(res))
        
    print(trans.kind, trans.length, results.shape[1])
    print('nans:', np.isnan(results).sum())

desc3D 2000 639
nans: 0
cats3D 2000 126
nans: 0
pharm3D 1024 1024
nans: 0
electroshape 2000 15
nans: 0
usr 12 12
nans: 0
usrcat 60 60
nans: 0


In [88]:
# trans = FPVecTransformer("usrcat", length=60, dtype=np.float32)
trans = FPVecTransformer("pharm3D", length=1024, dtype=np.float32)

results.shape



(7939, 1024)

In [83]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit_transform(res).shape

(7939, 60)

In [31]:

X_train = np.concatenate([train_ft, ohe.fit_transform(train[["prop"]])], axis=1)
X_train.shape

(7939, 9110)

In [32]:
eval_model('all stack', clf, X_train, y_train);

all stack: 0.8974    (0.905 ± 0.007)    204.7s


In [39]:
from src.config import mem
mord = FPVecTransformer("mordred", n_jobs=-1, dtype=np.float32)

md_feats = mem.cache(mord.transform)(train.smi)

In [60]:
from src.utils import drop_nans_non_unique
df = pd.DataFrame(md_feats)
na_cols = df.isna().sum() > 1
const_cols = df.nunique() == 1
md_mask = ~(na_cols | const_cols)

In [68]:
from src.corr import non_corr_features

uncorr = non_corr_features(df.loc[:, md_mask], y_train, threshold=0.95)
md_mask.sum(), uncorr.shape[1]

(1009, 497)

In [59]:
md_feats[:, md_mask].shape

(7939, 1009)

502

In [18]:
X_test = np.concatenate([test_ft, ohe.transform(test[["property"]].to_numpy())], axis=-1)

In [19]:
preds = clf.predict_proba(X_test)